In [2]:
agent_id = 'demo.node'
payload = {"data":"content"}
token =  {}


import os
if not os.path.exists("./"+agent_id.replace('.', '/')+".py"):
    print("./"+agent_id.replace('.', '/')+".py")

from importlib import reload , import_module
agt = import_module(agent_id)
reload(agt)

result = agt.pocketflow(payload,token)
result

SummarizeFile


{'data': 'content', 'summary': 'Summarize this text in 10 words: content'}

In [6]:
from importlib import reload , import_module
import MQ
reload(MQ)
#清空 Redis
#MQ.r.flushdb()


import requests

MQ_URL = "http://localhost:8000/MQ/"
TARGET = "MQ"
ENDS = ["Processing","Completed","Failed","BadFormat","Cancelled"]

for ending in ENDS:
    res = requests.post(MQ_URL+"send", json={
        "topic": "demo.node",
        "payload": {"data": "convert"},
        "target": TARGET,
        "resource": "tdd",
        "priority": 10
    })
    print(res.status_code,res.text,"\r\n")
    
    ### poll
    res = requests.get(MQ_URL+TARGET+"/poll")
    print(res.status_code,res.json(),"\r\n")
    JOB_ID = res.json().get("uuid")
    ### job
    res = requests.get(MQ_URL+TARGET+"/poll/"+JOB_ID)
    print(res.status_code,res.json(),"\r\n")
    ### ack
    res = requests.post(MQ_URL+TARGET+"/ack/"+JOB_ID)
    print(res.status_code,res.json(),"\r\n")
    ### query
    res = requests.get(MQ_URL+TARGET+"/query/"+JOB_ID)
    print(res.status_code,res.json(),"\r\n\n++++++++++++++status+++++++++++++++++\r\n")
    ### status
    for status in ["Processing",ending]:
        from datetime import datetime
        res = requests.post(MQ_URL+TARGET+"/status/", json={"uuid": JOB_ID, "status": status,"detail": {"stamp": datetime.now().isoformat()}})
        
        res = requests.get(MQ_URL+TARGET+"/query/"+JOB_ID)
        print(res.status_code,res.json(),"\r\n")

res = requests.get(MQ_URL+TARGET+"/stats")   
res.json()


200 {"uuid":"c91e1b27-97d6-4c79-a0c8-2670e7795b74"} 

200 {'uuid': 'c91e1b27-97d6-4c79-a0c8-2670e7795b74', 'topic': 'demo.node', 'payload': '{"data": "convert"}', 'target': 'MQ', 'resource': 'tdd', 'priority': '10', 'timestamp': '2025-08-21T12:25:49.387399+00:00', 'status': 'Pending'} 

200 {'uuid': 'c91e1b27-97d6-4c79-a0c8-2670e7795b74', 'topic': 'demo.node', 'payload': '{"data": "convert"}', 'target': 'MQ', 'resource': 'tdd', 'priority': '10', 'timestamp': '2025-08-21T12:25:49.387399+00:00', 'status': 'Pending'} 

200 {'status': 'acknowledged'} 

200 {'uuid': 'c91e1b27-97d6-4c79-a0c8-2670e7795b74', 'topic': 'demo.node', 'payload': '{"data": "convert"}', 'target': 'MQ', 'resource': 'tdd', 'priority': '100010', 'timestamp': '2025-08-21T12:25:49.387399+00:00', 'status': 'Acked'} 

++++++++++++++status+++++++++++++++++

200 {'uuid': 'c91e1b27-97d6-4c79-a0c8-2670e7795b74', 'topic': 'demo.node', 'payload': '{"data": "convert"}', 'target': 'MQ', 'resource': 'tdd', 'priority': '100010', 'tim

{'target': 'MQ',
 'stats': {'Pending': 0,
  'Acked': 0,
  'Started': 0,
  'Processing': 3,
  'Completed': 3,
  'Failed': 3,
  'BadFormat': 3,
  'Cancelled': 3}}

In [ ]:
from temporalio.client import Client
from temporalio.worker import Worker
from jobs import MQflow
from activities import RUN

client = await Client.connect("localhost:7233")
worker = Worker(
    client,
    task_queue="MQ",
    workflows=[MQflow],
    activities=[RUN],
    )
await worker.run()

200 {'uuid': '1cf50378-e309-4487-954f-ac4c856e74ea', 'topic': 'demo.node', 'payload': "{'data': 'convert'}", 'target': 'MQ', 'resource': 'tdd', 'priority': '10', 'timestamp': '2025-08-21T05:08:02.048150+00:00', 'status': 'Pending'}
200 {'uuid': '1cf50378-e309-4487-954f-ac4c856e74ea', 'topic': 'demo.node', 'payload': "{'data': 'convert'}", 'target': 'MQ', 'resource': 'tdd', 'priority': '10', 'timestamp': '2025-08-21T05:08:02.048150+00:00', 'status': 'Pending'}
200 {'uuid': '1cf50378-e309-4487-954f-ac4c856e74ea', 'topic': 'demo.node', 'payload': "{'data': 'convert'}", 'target': 'MQ', 'resource': 'tdd', 'priority': '10', 'timestamp': '2025-08-21T05:08:02.048150+00:00', 'status': 'Pending'}


In [7]:
import asyncio
import uuid
from temporalio.client import Client

from tqdm import tqdm
import requests
MQ_URL = "http://localhost:8000/MQ/"
TARGET = "MQ"
SIZE = 10

for i, o in enumerate(tqdm( range(SIZE*1) )):
    res = requests.post(MQ_URL+"send", json={
            "topic": "demo.node",
            "payload": {"data": "convert"},
            "target": TARGET,
            "resource": "tdd",
            "priority": 10
        })
    #print(res.status_code,res.json())

##
client = await Client.connect("localhost:7233")

# for i, o in enumerate(tqdm( range(SIZE) )):
#     await client.execute_workflow(
#         "MQflow",
#         {
#           # "uuid" : res.json()["uuid"]
#           # "topic":"demo.node",
#           # "input":{ "data":"hello" } 
#         },
#         id=f"ABC-{uuid.uuid4()}",
#         task_queue="MQ",
#     )
#     pass

ids = []
for i, o in enumerate(tqdm( range(SIZE) )):
    handle = await client.start_workflow(
        "MQflow",
        {
          # "uuid" : res.json()["uuid"]
          # "topic":"demo.node",
          # "input":{ "data":"hello" } 
        },
        id=f"ABC-{uuid.uuid4()}",
        task_queue="MQ",
    )
    ids.append(handle.id)
    pass

res = requests.get(MQ_URL+TARGET+"/stats")   
res.json()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 37.01it/s]


{'target': 'MQ',
 'stats': {'Pending': 9997,
  'Acked': 0,
  'Started': 0,
  'Processing': 6,
  'Completed': 3,
  'Failed': 3,
  'BadFormat': 3,
  'Cancelled': 3}}

In [10]:
res = requests.get(MQ_URL+TARGET+"/stats")   
res.json()

{'target': 'MQ',
 'stats': {'Pending': 9992,
  'Acked': 0,
  'Started': 0,
  'Processing': 3,
  'Completed': 11,
  'Failed': 3,
  'BadFormat': 3,
  'Cancelled': 3}}

In [2]:
from importlib import reload , import_module
from fastapi.testclient import TestClient

import MQ
reload(MQ)

import redis  #.asyncio as redis
r = redis.Redis(host="localhost", port=6379, db=9,decode_responses=True)
# ----------- Optional Test ----------- #
def run_test():
    client = TestClient(MQ.app)

    print("🚀 Running test with status enum...")

    
    res = client.post("/MQ/send", json={
        "topic": "topic.abc",
        "payload": {"cmd1": "convert"},
        "target": "worker1",
        "resource": "tdd",
        "priority": 10
    })

    res = client.post("/MQ/send", json={
        "topic": "topic.abc",
        "payload": {"cmd2": "convert"},
        "target": "worker1",
        "resource": "tdd",
        "priority": 10
    })
    
    assert res.status_code == 200
    uuid = res.json()["uuid"]
    # new_priority 优先级修改
    assert client.post(f"/MQ/priority", json={
                                            "uuid": uuid, 
                                            "target": "worker1",
                                            "new_priority":1
                                        }).status_code == 200

    nxt = client.get("/MQ/worker1/poll").json()
    print(uuid,'\r\n',nxt)
    
    # cancel mq 取消pendding的MQ
    assert client.post(f"/MQ/cancel", json={
                                            "uuid": uuid, 
                                            "target": "worker1"
                                        }).status_code == 200

    nxt = client.get("/MQ/worker1/poll").json()
    print(uuid,'\r\n',nxt)
    uuid = nxt["uuid"]
    #by id
    assert client.get(f"/MQ/worker1/poll/{uuid}").json()["uuid"] == uuid

    #ack
    assert client.post(f"/MQ/worker1/ack/{uuid}").status_code == 200

    #processing
    assert client.post(f"/MQ/worker1/status", json={"uuid": uuid, "status": "Processing"}).status_code == 200

    #complete
    assert client.post(f"/MQ/worker1/status", json={"uuid": uuid, "status": "Completed", "detail": {"out": "done.mp4"}}).status_code == 200
    
    #完成后队列会清理 404
    nxt = client.get(f"/worker1/poll/{uuid}")
    print(uuid,'\r\n',nxt)

    #
    assert client.get("/MQ/worker1/stats/topic.abc?status=Completed").json()["count"] == 0
    print("✅ Test passed!")

    client.get("/MQ/webhook").json()

#r.flushdb()
run_test()    

🚀 Running test with status enum...
Webhook 发送成功: 200
Webhook 发送成功: 200
dc3aebd7-9dbd-44f5-8ce5-9f17962552a7 
 {'uuid': 'dc3aebd7-9dbd-44f5-8ce5-9f17962552a7', 'topic': 'topic.abc', 'payload': "{'cmd2': 'convert'}", 'target': 'worker1', 'resource': 'tdd', 'priority': '1', 'timestamp': '2025-08-18T03:00:13.809204+00:00', 'status': 'Pending'}
Webhook 发送成功: 200
消息 dc3aebd7-9dbd-44f5-8ce5-9f17962552a7 因状态 Cancelled 被删除
dc3aebd7-9dbd-44f5-8ce5-9f17962552a7 
 {'uuid': '0f333f42-dedd-44be-8bd0-34ba7ea45851', 'topic': 'topic.abc', 'payload': "{'cmd1': 'convert'}", 'target': 'worker1', 'resource': 'tdd', 'priority': '10', 'timestamp': '2025-08-18T03:00:13.652640+00:00', 'status': 'Pending'}
优先级更新为：100010
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
消息 0f333f42-dedd-44be-8bd0-34ba7ea45851 因状态 Completed 被删除
0f333f42-dedd-44be-8bd0-34ba7ea45851 
 <Response [404 Not Found]>
✅ Test passed!
🚀 Webhook resend 开始 @ 44
[Webhook Retry] 重发：2becd171-fa10-4f65-8f9b-bc05919fd561
✅ Webhook 发送成功: webhoo

In [95]:
import concurrent.futures
from fastapi.testclient import TestClient
import random
import time
from importlib import reload, import_module

import MQ
reload(MQ)
client = TestClient(MQ.app)

# 清空 Redis
MQ.r.flushdb()

NUM_CLIENTS =10
TOPIC = "stress.topic"
TARGET = "worker1"

def simulate_client(i):
    # query is empty
    res = client.get(f"/MQ/{TARGET}/poll")
    if res.status_code == 200 and not res.json():
        print("✅ Queue is empty after all messages processed")

    # Send message
    res = client.post("/MQ/send", json={
        "topic": TOPIC,
        "payload": {"cmd": f"process-{i}"},
        "target": TARGET,
        "resource": "TDD",
        "priority": random.randint(1, 10)
    })
    if res.status_code != 200:
        return f"Client-{i} send failed"

    uuid = res.json()["uuid"]

    # Poll
    msg = client.get(f"/MQ/{TARGET}/poll/{uuid}").json()
    if not msg or msg["uuid"] != uuid:
        return f"Client-{i} poll mismatch"

    # ACK
    res = client.post(f"/MQ/{TARGET}/ack/{uuid}")
    if res.status_code != 200:
        return f"Client-{i} ack failed"

    # Status updates
    for status in ["Started", "Processing", "Completed"]:
        res = client.post(f"/MQ/{TARGET}/status", json={
            "uuid": uuid,
            "status": status,
            "detail": {"step": status}
        })
        if res.status_code != 200:
            return f"Client-{i} update {status} failed"
    
    return f"Client-{i} ✅ success"

def run_concurrent_test():
    print(f"🚀 Starting stress test with {NUM_CLIENTS} clients...")
    start = time.time()

    with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_CLIENTS) as executor:
        results = list(executor.map(simulate_client, range(NUM_CLIENTS)))

    for r in results:
        print(r)

    print(f"⏱️ Test completed in {time.time() - start:.2f}s")

run_concurrent_test()


🚀 Starting stress test with 10 clients...
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhoo